In [22]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from pydantic import BaseModel,Field
from langchain_core.tools import StructuredTool,tool,ToolException

In [23]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [24]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

In [25]:
@tool
def search(query:str) -> str:
    """
    Look up things online
    """
    return "Langchain"

In [26]:
print(search.name)
print(search.description)
print(search.args)

search
Look up things online
{'query': {'title': 'Query', 'type': 'string'}}


In [27]:
@tool
def multiply(a:int, b:int) -> int:
    """
    Multiply two numbers
    """
    return a*b

In [28]:
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
False


In [29]:
class SearchInput(BaseModel):
    query:str=Field(description="should be a search query")

@tool("search-tool",args_schema=SearchInput,return_direct=True)
def search(query:str) -> str:
    """
    Look up things online
    """
    return "Langchain"

In [30]:
print(search.name)
print(search.description)
print(search.args)
print(search.return_direct)

search-tool
Look up things online
{'query': {'description': 'should be a search query', 'title': 'Query', 'type': 'string'}}
True


<h3> StructuredTool DataClass <h3>

In [31]:
def searchFunction(query:str):
    return "Langchain"

search=StructuredTool.from_function(
    func=searchFunction,
    name="Search",
    description="Useful when you need to answer questions about current events"
)

In [32]:
print(search.name)
print(search.description)
print(search.args)

Search
Useful when you need to answer questions about current events
{'query': {'title': 'Query', 'type': 'string'}}


<h3> Using ArgSchema with StructuredTool <h3>

In [33]:
class Calculator(BaseModel):
    a:int=Field(description="first number")
    b:int=Field(description="second number")

def multiply(a:int, b:int) -> int:
    """
        Multiply two numbers
    """
    return a*b

In [34]:
calculator=StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="Multiply Numbers",
    args_schema=Calculator,
    return_direct=True
)

In [35]:
print(calculator.name)
print(calculator.description)
print(calculator.args)
print(calculator.return_direct)

Calculator
Multiply Numbers
{'a': {'description': 'first number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'integer'}}
True


In [37]:
calculator.invoke(input={'a':7,'b':6})

42

<h3> Handling Tool Errors </h3>

In [81]:
class Calculator(BaseModel):
    a:int=Field(description="first number")
    b:int=Field(description="second number")

def multiply(a:int, b:int) -> int:
    """
        Multiply two numbers less than 5
    """
    if a<5 and b<5:
        return a*b
    else:
        raise ToolException("Value of a and b should be less than 5")

calculator=StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="Multiply Numbers",
    args_schema=Calculator,
    return_direct=True,
    handle_tool_error=True  # This is a must for getting the error output
)

In [82]:
# calculator.invoke(input={'a':'a','b':6})
calculator.invoke(input={'a':6,'b':9})

'Value of a and b should be less than 5'

In [83]:
def search_tool1(s: str):
    try:
        return s/2
    except:
        raise ToolException("You cannot do this operation with a String")

In [84]:
search = StructuredTool.from_function(
    func=search_tool1,
    name="Search_tool1",
    description="A bad tool",
    handle_tool_error=True  # This is a must for getting the error output
)

search.invoke(input='65')

'You cannot do this operation with a String'